In [4]:
import pandas as pd


In [6]:
import pandas as pd

parsed = pd.read_csv(
    "../parsed_sample.csv",
    parse_dates=["timestamp"]
)

parsed.head()


,date,time,level,component,message,timestamp
0,81110,210118,INFO,dfs.FSNamesystem,BLOCK* NameSystem.delete: blk_-170857579237596...,2008-11-10 21:01:18
1,81111,53144,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-3552845605773...,2008-11-11 05:31:44
2,81111,51549,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,2008-11-11 05:15:49
3,81109,220522,WARN,dfs.DataNode$DataXceiver,10.250.14.38:50010:Got exception while serving...,2008-11-09 22:05:22
4,81110,132846,INFO,dfs.DataBlockScanner,Verification succeeded for blk_338667424734483...,2008-11-10 13:28:46


In [7]:
parsed.drop(columns=["date", "time"], inplace=True)
parsed.head()


,level,component,message,timestamp
0,INFO,dfs.FSNamesystem,BLOCK* NameSystem.delete: blk_-170857579237596...,2008-11-10 21:01:18
1,INFO,dfs.DataNode$PacketResponder,PacketResponder 2 for block blk_-3552845605773...,2008-11-11 05:31:44
2,INFO,dfs.FSNamesystem,BLOCK* NameSystem.addStoredBlock: blockMap upd...,2008-11-11 05:15:49
3,WARN,dfs.DataNode$DataXceiver,10.250.14.38:50010:Got exception while serving...,2008-11-09 22:05:22
4,INFO,dfs.DataBlockScanner,Verification succeeded for blk_338667424734483...,2008-11-10 13:28:46


In [11]:
parsed["is_warn"] = (parsed["level"] == "WARN").astype(int)
parsed["is_info"] = (parsed["level"] == "INFO").astype(int)

parsed["msg_length"] = parsed["message"].str.len()



In [12]:
WINDOW_SIZE = 100   # 100 logs = 1 window
parsed["window"] = parsed.index // WINDOW_SIZE


In [15]:
features = parsed.groupby("window").agg({
    "is_warn": "sum",
    "is_info": "sum",
    "msg_length": "mean",
    "component": "nunique"
})

features.rename(
    columns={"component": "unique_components"},
    inplace=True
)

features.head()


,is_warn,is_info,msg_length,unique_components
window,,,,
0,5,95,93.37,5
1,4,96,94.14,5
2,3,97,92.40,4
3,2,98,89.93,5
4,4,96,91.04,4


In [16]:
features.to_csv("../features.csv", index=False)

